In [1]:
# load the iris dataset
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Load iris dataset
iris_data = datasets.load_iris()
iris = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris['type'] = iris_data.target.astype('int')
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [2]:
# Use all four features
X = iris.iloc[:, 0:4]
y = iris['type']

In [3]:
#Create a new feature by multiplying sepal length by sepal width and dividing by petal length * petal width. 

iris['new']=(iris.iloc[:,0]*iris.iloc[:,1])/(iris.iloc[:,2]*iris.iloc[:,3])  

In [4]:
# Sample 80% of the data for a training set stratifying on ‘type’ 

from sklearn.model_selection import train_test_split as tts 
X_train, X_test, y_train, y_test = tts(iris.iloc[:,0:3], iris.iloc[:,4], test_size=0.2, random_state=42,
stratify=iris.iloc[:,4]) 

In [5]:
print(X_train.columns)

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)'], dtype='object')


In [6]:
# Re-import necessary libraries after code execution state reset
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Load iris dataset
iris_data = datasets.load_iris()
iris = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris['type'] = iris_data.target.astype('int')

# Use all four features
X = iris.iloc[:, 0:4]
y = iris['type']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


#On the test set, evaluate the following two estimators for sepal width using ME, MPE, MAPE, MAE, and  MSE. 
#Mean of petal length calculated only on the training data 

#Mean of sepal length minus petal width calculated only on the training data 

# Define evaluation function
def myf(actual, predicted):
    actual = np.array(actual)
    predicted = np.array(predicted)
    me = np.mean(actual - predicted)
    mpe = np.mean((actual - predicted) / actual) * 100
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    mae = np.mean(np.abs(actual - predicted))
    mse = np.mean((actual - predicted) ** 2)

    return {
        "ME": round(me, 4),
        "MPE (%)": round(mpe, 4),
        "MAPE (%)": round(mape, 4),
        "MAE": round(mae, 4),
        "MSE": round(mse, 4)
    }

# Estimator 1: Mean of petal length (from training data)
est1_value = np.mean(X_train['petal length (cm)'])
est1 = [est1_value] * len(y_test)

# Estimator 2: Mean of (sepal length - petal width) (from training data)
est2_value = np.mean(X_train['sepal length (cm)'] - X_train['petal width (cm)'])
est2 = [est2_value] * len(y_test)

# Evaluate on actual sepal width values
metrics_est1 = myf(X_test['sepal width (cm)'], est1)
metrics_est2 = myf(X_test['sepal width (cm)'], est2)

# Show comparison
results_df = pd.DataFrame([metrics_est1, metrics_est2], index=["Estimator 1", "Estimator 2"])
print(results_df)


                 ME  MPE (%)  MAPE (%)     MAE     MSE
Estimator 1 -0.6767 -23.7499   24.1859  0.6940  0.6018
Estimator 2 -1.5433 -52.1981   52.1981  1.5433  2.5258


In [7]:
from numpy import percentile
import numpy as np
from sklearn.metrics import classification_report as cr

# First classifier: thresholds at 25th and 50th percentiles
est3 = percentile(X_train['sepal length (cm)'], [25, 50])

y_hat = np.zeros(len(y_test))
y_hat[X_test['sepal length (cm)'] > est3[0]] = 1
y_hat[X_test['sepal length (cm)'] > est3[1]] = 2
y_hat = y_hat.astype(int)

print("Classifier 1 (25th, 50th percentile thresholds):")
print(cr(y_test.astype(int), y_hat))
print("-" * 50)

# Second classifier: thresholds at 50th and 75th percentiles
est4 = percentile(X_train['sepal length (cm)'], [50, 75])

y_hat2 = np.zeros(len(y_test))
y_hat2[X_test['sepal length (cm)'] > est4[0]] = 1
y_hat2[X_test['sepal length (cm)'] > est4[1]] = 2
y_hat2 = y_hat2.astype(int)

print("Classifier 2 (50th, 75th percentile thresholds):")
print(cr(y_test.astype(int), y_hat2))


Classifier 1 (25th, 50th percentile thresholds):
              precision    recall  f1-score   support

           0       0.71      0.50      0.59        10
           1       0.33      0.20      0.25        10
           2       0.59      1.00      0.74        10

    accuracy                           0.57        30
   macro avg       0.55      0.57      0.53        30
weighted avg       0.55      0.57      0.53        30

--------------------------------------------------
Classifier 2 (50th, 75th percentile thresholds):
              precision    recall  f1-score   support

           0       0.69      0.90      0.78        10
           1       0.38      0.30      0.33        10
           2       0.67      0.60      0.63        10

    accuracy                           0.60        30
   macro avg       0.58      0.60      0.58        30
weighted avg       0.58      0.60      0.58        30

